### 01 packages

In [14]:
######################################################################################################
######################################################################################################

import pandas as pd

import numpy as np

######################################################################################################
######################################################################################################

import geopandas as gp

from shapely.geometry import Point, Polygon

import networkx as nx

import folium

import h3

######################################################################################################
######################################################################################################

import math

import random

import pulp

######################################################################################################
######################################################################################################

import copy

######################################################################################################
######################################################################################################

import scipy.stats as stats

######################################################################################################
######################################################################################################


from model.Matching import Bipartite_matching

from scipy.optimize import minimize

from tqdm import tqdm

######################################################################################################
######################################################################################################

import warnings

warnings.filterwarnings('ignore')

def Truncated_Gauss(mu,sigma):
    
    lower,upper=mu-sigma,mu+sigma

    X = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
    
    return round(X.rvs(1)[0],2)

######################################################################################################
######################################################################################################

from tabulate import tabulate

def show_df(df):

    print(tabulate(df,numalign = 'center', stralign = 'center', tablefmt = "grid", showindex = True,headers=df.columns))


### 02 parameter

In [15]:
scenario='03PIFCT_vs_Static'

######################################################################################################
######################################################################################################

day=1

start_hour=7

simulation_duration= 1 # [hour]

start_stamp=start_hour*3600

######################################################################################################
######################################################################################################

batch_interval=15 # [second]

######################################################################################################
######################################################################################################

dispatching_radius=5 # [km]

######################################################################################################
######################################################################################################

speed=20000/60 # [m/min]

######################################################################################################
######################################################################################################

waiting_patience=180 # [s]


######################################################################################################
######################################################################################################

alpha_mu_0,alpha_sigma_0 = 0.0,0.05

alpha_mu_1,alpha_sigma_1 = 3.2,0.1

alpha_mu_2,alpha_sigma_2 = 0.6,0.05

u_decline_mu,u_decline_sigma = -13.2,0.5

######################################################################################################
######################################################################################################

beta_mu_0,beta_sigma_0 = 0.0,0.05

beta_mu_1,beta_sigma_1 = 2.0,0.2

beta_mu_2,beta_sigma_2 = 0.5,0.05

v_decline_mu,v_decline_sigma=8.5,0.5

######################################################################################################
######################################################################################################

actual_theta_0=2.75

actual_theta_1=0.37

actual_theta_2=1.05

actual_lambda_=0.78

### 03 joint strategy

In [18]:
######################################################################################################
######################################################################################################

def static(etd_matrix,ett_matrix,tau_matrix,theta_0,theta_1,theta_2,lambda_):

    ################################################################################################

    order_num,driver_num=etd_matrix.shape[0],etd_matrix.shape[1]
    
    ################################################################################################

    fare=theta_0*np.ones([order_num,driver_num])+theta_1*etd_matrix+theta_2*ett_matrix

    wage=fare*lambda_
    
    ################################################################################################
    
    weight=np.zeros([order_num,driver_num])
    
    for o in range(order_num):

        for d in range(driver_num):
            
            if tau_matrix[o,d]<dispatching_radius:

                weight[o,d]=(fare[o,d]-wage[o,d])/tau_matrix[o,d]

    x=Bipartite_matching(weight,tau_matrix)

    return fare,wage,x

######################################################################################################
######################################################################################################

def expected_profit(x,tau,u0,v0,u2,v2):

    u1=alpha_mu_0-alpha_mu_1*x[0]-alpha_mu_2*tau

    v1=beta_mu_0+beta_mu_1*x[1]-beta_mu_2*tau
    
    ################################################################################################

    p=np.exp(u1)/(np.exp(u0)+np.exp(u2)+np.exp(u1))

    q=np.exp(v1)/(np.exp(v0)+np.exp(v2)+np.exp(v1))

    ################################################################################################

    return -1*p*q*(x[0]-x[1])

def PIFCT(etd_matrix,ett_matrix,tau_matrix):
    
    ################################################################################################
    
    estimated_theta_0=actual_theta_0

    estimated_theta_1=actual_theta_1

    estimated_theta_2=actual_theta_2

    estimated_lambda_=actual_lambda_
        
    fare2,wage2,x2=static(etd_matrix,ett_matrix,tau_matrix,\
                          estimated_theta_0,estimated_theta_1,estimated_theta_2,estimated_lambda_)
    
    ################################################################################################

    order_num,driver_num=etd_matrix.shape[0],etd_matrix.shape[1]
    
    ################################################################################################

    u2_matrix=alpha_mu_0*np.ones([order_num,driver_num])-alpha_mu_1*fare2-alpha_mu_2*tau_matrix

    v2_matrix=beta_mu_0*np.ones([order_num,driver_num])+beta_mu_1*wage2-beta_mu_2*tau_matrix
        
    ################################################################################################
        
    matched_order_driver_pair={}

    matched_driver_order_pair={}

    for o,d in zip(np.nonzero(x2)[0],np.nonzero(x2)[1]):

        matched_order_driver_pair[o]=d

        matched_driver_order_pair[d]=o

    
    ################################################################################################

    fare1,wage1,weight1=float('inf')*np.ones([order_num,driver_num]),np.zeros([order_num,driver_num]),np.zeros([order_num,driver_num])

    for o in range(order_num):

        for d in range(driver_num):
            
            if tau_matrix[o,d]<dispatching_radius:
                
                # calculate the input
                
                if o in matched_order_driver_pair.keys():
                
                    matched_d=matched_order_driver_pair[o]
                    
                    u2=u2_matrix[o,matched_d]
                    
                else:
                    
                    u2=-float('inf')
                    
                if d in matched_driver_order_pair.keys():
                
                    matched_o=matched_driver_order_pair[d]
                    
                    v2=v2_matrix[matched_o,d]
                    
                else:
                    
                    v2=-float('inf')
                    
                tau=tau_matrix[o,d]
                
                u0=u_decline_mu*etd_matrix[o,d]

                v0=v_decline_mu
                
                # solve the fare and wage optimization

                fw_opt = minimize(expected_profit,[fare2[o,d],wage2[o,d]],args=(tau,u0,v0,u2,v2),bounds=((0, None), (0, None)))

                fare1[o,d]=fw_opt.x[0]

                wage1[o,d]=fw_opt.x[1]
                
                # calculate the weight in dispatching

                # option (i):(fare1[o,d]-wage1[o,d])/tau_matrix[o,d]
                
                # option (ii):-1*expected_profit(fw_opt.x,o,d,tau_matrix[o,d],u0,v0,u2[o,d],v2[o,d])

                weight1[o,d]=-1*expected_profit(fw_opt.x,tau,u0,v0,u2,v2)
    
    ################################################################################################

    x1=Bipartite_matching(weight1,tau_matrix)

    return fare1,wage1,x1

### 04 simulation

In [19]:
######################################################################################################
######################################################################################################


time_stamp=copy.copy(start_stamp+batch_interval)

######################################################################################################
######################################################################################################

# 01 customer demand data

order_df=pd.read_csv('./01data/01demand/customer_2023-02-'+str(day)+'.csv')

order_df=order_df.drop(columns=['Unnamed: 0'])

order_df=order_df.loc[order_df.hour==start_hour]

order_df=order_df.sort_values(by=['request_second'])

order_df=order_df.reset_index(drop=True)

######################################################################################################

order_df['driver_id']='waiting'

order_df['platform']=0

order_df['fare']=0.0

order_df['profit']=0.0

order_df['matching_second']=0

######################################################################################################
######################################################################################################

# 02 supply driver data

driver_df=pd.read_csv('./01data/02supply/supply_2023-02-'+str(day)+'.csv')

driver_df=driver_df.drop(columns=['Unnamed: 0'])

driver_df['second']=time_stamp

######################################################################################################

driver_df['order_id']='idle'

driver_df['platform']=0

driver_df['wage']=0.0

######################################################################################################
######################################################################################################

class Objects:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

# 03 loop simulation

record_df=pd.DataFrame([],columns=['second', 'Cul_matchings_1', 'Cul_profits_1', 'Cur_matchings_1', 'Cur_profits_1', 'Cul_matchings_2', 'Cul_profits_2', 'Cur_matchings_2', 'Cur_profits_2'])

while True:
    
    ######################################################################################################
    
    # 03-01 avaiable customer demand
    
    order_batch=order_df[(order_df['request_second']<=time_stamp)&(order_df['driver_id']=='waiting')]
    
    order_batch=order_batch.reset_index(drop=True)
    
    order_batch_info=list()
    
    for idx,row in order_batch.iterrows():
    
        order_batch_info.append(Objects(**row.to_dict()))
        
    ######################################################################################################
    
    # 03-02 avaiable driver supply
    
    driver_batch=driver_df[(driver_df['second']==time_stamp)&(driver_df['order_id']=='idle')]
    
    driver_batch=driver_batch.reset_index(drop=True)
    
    driver_batch_info=list()
    
    for idx,row in driver_batch.iterrows():
    
        driver_batch_info.append(Objects(**row.to_dict()))
        
    ######################################################################################################
    
    # 03-03 calcualte the distance and time matrix
    
    order_num,driver_num=order_batch.shape[0],driver_batch.shape[0]
    
    etd_matrix=np.ones([order_num,driver_num])
    
    ett_matrix=np.ones([order_num,driver_num])
    
    tau_matrix=np.ones([order_num,driver_num])
    
    for i in range(order_num):
        
        for j in range(driver_num):
            
            etd_matrix[i,j]=order_batch_info[i].distance/1000.0 # [km]
            
            ett_matrix[i,j]=order_batch_info[i].distance/speed # [min]
            
            pick_up_distance=Point(order_batch_info[i].pickup_lng,order_batch_info[i].pickup_lat).distance(Point(driver_batch_info[j].lng,driver_batch_info[j].lat))*110000*1.2

            tau_matrix[i,j]=pick_up_distance/speed # [min]
            
    ######################################################################################################
    
    # 03-04 platform 2 offers the dispatching and pricing
    
    fare2,wage2,x2=static(etd_matrix,ett_matrix,tau_matrix,\
                          actual_theta_0,actual_theta_1,actual_theta_2,actual_lambda_)
    
    ######################################################################################################
    
    # 03-05 platform 1 offers the dispatching and pricing
    
    fare1,wage1,x1=PIFCT(etd_matrix,ett_matrix,tau_matrix)
    
    ######################################################################################################
    
    # 03-06 customers make a choice
    
    alpha_0_matrix=np.array(order_batch.alpha_0.to_list())
    
    alpha_0_matrix=np.tile(np.array([alpha_0_matrix]).transpose(), (1,driver_num))

    alpha_1_matrix=np.array(order_batch.alpha_1.to_list())
    
    alpha_1_matrix=np.tile(np.array([alpha_1_matrix]).transpose(), (1,driver_num))
    
    alpha_2_matrix=np.array(order_batch.alpha_2.to_list())
    
    alpha_2_matrix=np.tile(np.array([alpha_2_matrix]).transpose(), (1,driver_num))
    
    u1=alpha_0_matrix*np.ones([order_num,driver_num])-alpha_1_matrix*fare1-alpha_2_matrix*tau_matrix

    u2=alpha_0_matrix*np.ones([order_num,driver_num])-alpha_1_matrix*fare2-alpha_2_matrix*tau_matrix
    
    order_selection={}
    
    for i  in range(order_num):
        
        choice_set={0:order_batch_info[i].u_decline}
        
        matching1,matching2=np.nonzero(x1[i,:])[0],np.nonzero(x2[i,:])[0]
        
        if len(matching1)>0:
        
            d1=matching1[0]
            
            choice_set[1]=u1[i,d1]
            
        if len(matching2)>0:
            
            d2=matching2[0]
            
            choice_set[2]=u2[i,d2]
            
        choice_prob={}
        
        exp_sum=sum([np.exp(u) for u in choice_set.values()])
        
        for select,utility in choice_set.items():
            
            choice_prob[select]=np.exp(utility)/exp_sum
        
        selection=np.random.choice(list(choice_prob.keys()),p=list(choice_prob.values()))
        
        if selection==1:
            
            selection=str(selection)+'_'+str(matching1[0])
            
        elif selection==2:
            
            selection=str(selection)+'_'+str(matching2[0])
            
        else:
            
            selection=str(selection)+'_null'
            
        order_selection[i]=selection
        
    ######################################################################################################
    
    # 03-07 drivers make a choice
    
    beta_0_matrix=np.array(driver_batch.beta_0.to_list())
    
    beta_0_matrix = np.tile(beta_0_matrix, (order_num, 1))
    
    beta_1_matrix=np.array(driver_batch.beta_1.to_list())
    
    beta_1_matrix = np.tile(beta_1_matrix, (order_num, 1))
    
    beta_2_matrix=np.array(driver_batch.beta_2.to_list())
    
    beta_2_matrix = np.tile(beta_2_matrix, (order_num, 1))
    
    v1=beta_0_matrix*np.ones([order_num,driver_num])+beta_1_matrix*wage1-beta_2_matrix*tau_matrix
    
    v2=beta_0_matrix*np.ones([order_num,driver_num])+beta_1_matrix*wage2-beta_2_matrix*tau_matrix
    
    driver_selection={}
    
    for j  in range(driver_num):
        
        choice_set={0:driver_batch_info[j].v_decline}
        
        matching1,matching2=np.nonzero(x1[:,j])[0],np.nonzero(x2[:,j])[0]
        
        if len(matching1)>0:
        
            o1=matching1[0]
            
            choice_set[1]=v1[o1,j]
            
        if len(matching2)>0:
            
            o2=matching2[0]
            
            choice_set[2]=v2[o2,j]
            
        choice_prob={}
        
        exp_sum=sum([np.exp(u) for u in choice_set.values()])
        
        for select,utility in choice_set.items():
            
            choice_prob[select]=np.exp(utility)/exp_sum
            
        selection=np.random.choice(list(choice_prob.keys()),p=list(choice_prob.values()))
        
        if selection==1:
            
            selection=str(selection)+'_'+str(matching1[0])
            
        elif selection==2:
            
            selection=str(selection)+'_'+str(matching2[0])
            
        else:
            
            selection=str(selection)+'_null'
            
        driver_selection[j]=selection
        
    ######################################################################################################
    
    # 03-08 matching in the market
    
    matchings={1:[],2:[]}
    
    profits={1:0.0,2:0.0}
    
    for o,order_select in order_selection.items():
        
        order_platform,d=int(order_select.split('_')[0]),order_select.split('_')[1]
        
        if order_platform:
            
            d=int(d)
            
            driver_select=driver_selection[d]
            
            driver_platform=int(driver_select.split('_')[0])
            
            if driver_platform==order_platform:
                
                matchings[driver_platform].append((o,d))
                
                if driver_platform==1:
                    
                    profits[driver_platform]+=fare1[o,d]-wage1[o,d]
                    
                elif driver_platform==2:
                    
                    profits[driver_platform]+=fare2[o,d]-wage2[o,d]
                
    ######################################################################################################
    
    # 03-09 update the information of demand and supply
    
    for p,matching in matchings.items():
        
        if len(matching)!=0:
            
            for pair in matching:
                
                o,d=pair[0],pair[1]
                
                order_id,driver_id=order_batch_info[o].order_id,driver_batch_info[d].driver_id
                
                ######################################################################################################
                
                if p==1:
                    
                    fare=fare1[o,d]
                    
                    wage=wage1[o,d]
                    
                elif p==2:
                    
                    fare=fare2[o,d]
                    
                    wage=wage2[o,d]
                
                ######################################################################################################
                
                order_df.loc[(order_df['order_id']==order_id),'driver_id']=driver_id
                
                order_df.loc[(order_df['order_id']==order_id),'platform']=p
                
                order_df.loc[(order_df['order_id']==order_id),'fare']=fare
                
                order_df.loc[(order_df['order_id']==order_id),'profit']=fare-wage
                
                order_df.loc[(order_df['order_id']==order_id),'matching_second']=time_stamp

                ######################################################################################################

                driver_df.loc[(driver_df['second']==time_stamp)&(driver_df['driver_id']==driver_id),'order_id']=order_id
                
                driver_df.loc[(driver_df['second']==time_stamp)&(driver_df['driver_id']==driver_id),'wage']=wage
                
                driver_df.loc[(driver_df['second']==time_stamp)&(driver_df['driver_id']==driver_id),'platform']=p
                
                ######################################################################################################

                added_item={}

                added_item['driver_id']=driver_id

                added_item['order_id']='idle'
                
                added_item['platform']=0

                added_item['second']=time_stamp+int(tau_matrix[o,d]*60)+int(ett_matrix[o,d]*60)
                
                added_item['lat']=order_batch_info[o].dropoff_lat
                
                added_item['lng']=order_batch_info[o].dropoff_lng
                
                added_item['beta_0']=driver_batch_info[d].beta_0

                added_item['beta_1']=driver_batch_info[d].beta_1

                added_item['beta_2']=driver_batch_info[d].beta_2

                added_item['v_decline']=driver_batch_info[d].v_decline
                
                added_item['wage']=0.0
                
                driver_df=driver_df.append(added_item, ignore_index=True)
                
    ######################################################################################################
    
    # 03-10 repositioning
    
    repositioned_drivers=copy.deepcopy(driver_df.loc[(driver_df['second']==time_stamp)&(driver_df['order_id']=='idle')])

    repositioned_drivers['second']+=batch_interval

    driver_df=pd.concat([driver_df,repositioned_drivers],ignore_index=True)
    
    ######################################################################################################
    
    # 03-11 customer cancellation
    
    unmatched_orders=order_df[(order_df['request_second']<=time_stamp)&(order_df['driver_id']=='waiting')].order_id.to_list()

    if len(unmatched_orders)!=0:

        order_df.loc[(time_stamp-order_df['request_second']>=waiting_patience)&(order_df['order_id'].isin(unmatched_orders)),'driver_id']='cancelled'
        
    ######################################################################################################
    
    # 03-12 dynamically report the performance
    
    order_df1=order_df.loc[(order_df.platform==1)]

    order_df2=order_df.loc[(order_df.platform==2)]

    matchings1=order_df1.order_id.count()

    matchings2=order_df2.order_id.count()

    profit1=order_df1.profit.sum()

    profit2=order_df2.profit.sum()

    indexes=['Time',\
            'Culmulative matchings',\
            'Culmulative profit',\
            'Current matchings',\
            'Current profit']

            

    df=pd.DataFrame(indexes,columns=['indexes'])

    values1=[time_stamp,\
             matchings1,profit1,\
             len(matchings[1]),profits[1]]

    values2=[time_stamp,\
             matchings2,profit2,\
             len(matchings[2]),profits[2]]

    df['1']=values1

    df['2']=values2
    
    print('\n')
    print('*'*100)
    print('*'*100)

    show_df(df)
    
    ######################################################################################################
    
    # 03-13 dynamically save the performance
    
    added_item={}

    added_item['second']=int(time_stamp)

    added_item['Cul_matchings_1']=matchings1

    added_item['Cul_profits_1']=profit1
    
    added_item['Cur_matchings_1']=len(matchings[1])

    added_item['Cur_profits_1']=profits[1]

    added_item['Cul_matchings_2']=matchings2

    added_item['Cul_profits_2']=profit2
    
    added_item['Cur_matchings_2']=len(matchings[2])

    added_item['Cur_profits_2']=profits[2]
    
    record_df=record_df.append(added_item, ignore_index=True)
                
    ######################################################################################################
    
    # 03-14 dynamically update the timestamp
    
    time_stamp+=batch_interval
    
    ######################################################################################################
    
    # 03-15 quit condition
    
    completion_percentage=100*(order_df.loc[order_df.driver_id!='waiting'].shape[0])/(order_df.shape[0])
    
    print("Current completion percentage = %.1f"%completion_percentage)
    
    order_batch=order_df.loc[order_df.driver_id=='waiting']
    
    if order_batch.shape[0]==0:
        
        break
    

order_df.to_csv('./02result/'+scenario+'/customer_2023-02-'+str(day)+'.csv')

driver_df.to_csv('./02result/'+scenario+'/supply_2023-02-'+str(day)+'.csv')

record_df.to_csv('./02result/'+scenario+'/record_2023-02-'+str(day)+'.csv')




****************************************************************************************************
****************************************************************************************************
+----+-----------------------+---------+---------+
|    |        indexes        |    1    |    2    |
+====+=======================+=========+=========+
| 0  |         Time          |  25215  |  25215  |
+----+-----------------------+---------+---------+
| 1  | Culmulative matchings |    2    |    4    |
+----+-----------------------+---------+---------+
| 2  |  Culmulative profit   | 10.1719 | 25.6686 |
+----+-----------------------+---------+---------+
| 3  |   Current matchings   |    2    |    4    |
+----+-----------------------+---------+---------+
| 4  |    Current profit     | 10.1719 | 25.6686 |
+----+-----------------------+---------+---------+
Current completion percentage = 0.2


KeyboardInterrupt: 